In [54]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from docx import Document
from docx.shared import Cm

# 1. Pobieramy stronę z Wikipedii
strona = requests.get("https://en.wikipedia.org/wiki/Academy_Award_for_Best_Picture")
zupa = BeautifulSoup(strona.text, "html.parser")

# 2. Zbieramy tabelki ze zwycięzcami
tables = zupa.find_all("table")[2:13]

# 3. Łączymy wszystkie tabele w jedną
dataframes = [pd.read_html(str(t), flavor="lxml")[0] for t in tables]
tabela = pd.concat(dataframes)
tabela = tabela.dropna(subset=["Film"])

# 4. Wyciągamy listę zwycięzców i lata
Rok = "Przykład"
Rok_nagrody, Lista_zwycięzców = [], []
for i, row in enumerate(tabela.iloc[:, 0]):
    if row != Rok:
        Lista_zwycięzców.append(tabela.Film.iloc[i])
        Rok_nagrody.append(Rok)
        Rok = row

Rok_nagrody = Rok_nagrody[1:]
Rok_nagrody.append("2023")  # ostatni rok ręcznie

Poprawiony_rok_nagrody = []
for r in Rok_nagrody:
    znak = r.find("(")
    if znak != -1:
        r = r[:znak]
    Poprawiony_rok_nagrody.append(r.strip())

# 5. Zbieramy linki bezpośrednio z tabel (dokładne dopasowanie)
linki = []
for t in tables:
    for a in t.find_all("a"):
        href = a.get("href", "")
        if href.startswith("/wiki/") and "film" in href:
            linki.append("https://en.wikipedia.org" + href)

# Usuwamy duplikaty, zostawiając kolejność
seen = set()
pełne_linki = []
for l in linki:
    if l not in seen:
        pełne_linki.append(l)
        seen.add(l)

# Dopasowujemy do liczby filmów
pełne_linki = pełne_linki[:len(Lista_zwycięzców)]

# 6. Budujemy DataFrame
Filmy = pd.DataFrame({
    "Tytuł": Lista_zwycięzców,
    "Rok_wygrania_Oscara": Poprawiony_rok_nagrody,
    "Link": pełne_linki
})

# 7. Pobieramy reżysera
reżyserzy = []
for url in Filmy.Link:
    strona_f = requests.get(url)
    zupa_f = BeautifulSoup(strona_f.text, "html.parser")
    scrapy = zupa_f.find_all("td", attrs={"class": "infobox-data"})
    if scrapy:
        reżyserzy.append(scrapy[0].text.strip())
    else:
        reżyserzy.append("Brak danych")

Filmy["Reżyser"] = reżyserzy

# 8. Usuwamy kolumnę Link i zachowujemy tylko potrzebne kolumny
Filmy = Filmy[["Tytuł", "Rok_wygrania_Oscara", "Reżyser"]]

# 9. Zapis do Worda
document = Document()
style = document.styles["Normal"]
style.font.name = "Calibri"

for _, row in Filmy.iterrows():
    document.add_paragraph("Tytuł: " + row["Tytuł"])
    document.add_paragraph("Rok wygrania Oskara: " + row["Rok_wygrania_Oscara"])
    document.add_paragraph("Reżyser: " + row["Reżyser"])
    document.add_paragraph("")  # odstęp

document.save("Opis_Filmów.docx")

C:\Windows\Temp\ipykernel_20040\1028885929.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dataframes = [pd.read_html(str(t), flavor="lxml")[0] for t in tables]
C:\Windows\Temp\ipykernel_20040\1028885929.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dataframes = [pd.read_html(str(t), flavor="lxml")[0] for t in tables]
C:\Windows\Temp\ipykernel_20040\1028885929.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dataframes = [pd.read_html(str(t), flavor="lxml")[0] for t in tables]
C:\Windows\Temp\ipykernel_20040\1028885929.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be 